# 前海征信“好信杯”大数据算法赛（LB top20方案研究）

*赛题：https://www.kesci.com/apps/home/competition/58e46b3b9ed26b1e09bfbbb7/works*  
*参考：https://www.kesci.com/apps/home/project/59ca5ff521100106623f3db3 *  
注意：题目本意是想要做一些迁移学习内容，但这里仅在数据集B上进行数据的分析和测试 

 
### 内容包括：数据初探 + 数据探索性分析 + 特征工程 + 单模型训练


### 1、数据初探

In [7]:

# 导入需要的包
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb

In [8]:
# read data
train_A = pd.read_csv('A_train.csv')
train_B = pd.read_csv('B_train.csv')
test_B = pd.read_csv('B_test.csv')

In [16]:
# 查看数据集B的基本信息

train_B.info()
train_B.head()

# 发现：
#  数据集B一共有491个特征,同时数据中还含有大量的缺失值
#  数据的range很大,有5226.59的又有0的
#  数据主要由float64和int64两种数据


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Columns: 491 entries, no to UserInfo_270
dtypes: float64(488), int64(3)
memory usage: 15.0 MB


,no,UserInfo_1,UserInfo_2,UserInfo_3,UserInfo_4,ProductInfo_1,UserInfo_5,UserInfo_6,ProductInfo_2,UserInfo_7,...,UserInfo_264,UserInfo_265,ProductInfo_214,UserInfo_266,UserInfo_267,ProductInfo_215,ProductInfo_216,UserInfo_268,UserInfo_269,UserInfo_270
0,8192,NaN,5226.59,0.0,0.0,0.0,296.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,15.0,0.0,0.0,0.0,2940.0,NaN
1,1,NaN,0.00,0.0,0.0,0.0,NaN,0.0,0.0,46000.0,...,0.0,3.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN
2,8195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# 统计信息

pd.set_option('display.max_columns',200)
train_B.describe()

# 发现：
#   特征之间存在不同情况的缺失,例如：UserInfo_1中就只有一个数据,UserInfo_3中有1515个数据,缺失情况较为严重,UserInfo_12则不存在缺失情况. 而针对不同的缺失情况我们需要考虑删除特征或者样本删除的操作[1].
#   特征没有全部进行归一化,例如UserInfo_270的最小值为7000,最大的是401000,ProductInfo_216最小值为0,最大值为1,不同特征的range大小不一样. 如果希望尝试使用非基于树的模型,就不得不考虑Normalization操作了[2].
#   很多数据存在非常大的波动,例如UserInfo_3的std为11156.96,考虑是否需要进行特殊处理,本文暂不做处理[3].


,no,UserInfo_1,UserInfo_2,UserInfo_3,UserInfo_4,ProductInfo_1,UserInfo_5,UserInfo_6,ProductInfo_2,UserInfo_7,ProductInfo_3,ProductInfo_4,UserInfo_8,UserInfo_9,ProductInfo_5,ProductInfo_6,UserInfo_10,UserInfo_11,ProductInfo_7,UserInfo_12,ProductInfo_8,ProductInfo_9,ProductInfo_10,UserInfo_13,ProductInfo_11,ProductInfo_12,UserInfo_14,ProductInfo_13,ProductInfo_14,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_18,UserInfo_19,UserInfo_20,UserInfo_21,UserInfo_22,UserInfo_23,UserInfo_24,UserInfo_25,ProductInfo_15,ProductInfo_16,ProductInfo_17,UserInfo_26,ProductInfo_18,ProductInfo_19,UserInfo_27,ProductInfo_20,UserInfo_28,ProductInfo_21,ProductInfo_22,ProductInfo_23,UserInfo_29,UserInfo_30,UserInfo_31,UserInfo_32,UserInfo_33,ProductInfo_24,UserInfo_34,ProductInfo_25,ProductInfo_26,ProductInfo_27,ProductInfo_28,ProductInfo_29,ProductInfo_30,ProductInfo_31,UserInfo_35,ProductInfo_32,ProductInfo_33,UserInfo_36,UserInfo_37,ProductInfo_34,UserInfo_38,UserInfo_39,UserInfo_40,ProductInfo_35,ProductInfo_36,UserInfo_41,ProductInfo_37,UserInfo_42,UserInfo_43,UserInfo_44,UserInfo_45,ProductInfo_38,UserInfo_46,UserInfo_47,UserInfo_48,ProductInfo_39,UserInfo_49,UserInfo_50,UserInfo_51,ProductInfo_40,ProductInfo_41,UserInfo_52,ProductInfo_42,ProductInfo_43,ProductInfo_44,UserInfo_53,UserInfo_54,UserInfo_55,...,ProductInfo_168,UserInfo_220,ProductInfo_169,UserInfo_221,ProductInfo_170,ProductInfo_171,UserInfo_222,ProductInfo_172,UserInfo_223,ProductInfo_173,UserInfo_224,ProductInfo_174,UserInfo_225,UserInfo_226,UserInfo_227,ProductInfo_175,UserInfo_228,ProductInfo_176,UserInfo_229,ProductInfo_177,UserInfo_230,ProductInfo_178,UserInfo_231,UserInfo_232,UserInfo_233,UserInfo_234,ProductInfo_179,ProductInfo_180,UserInfo_235,ProductInfo_181,UserInfo_236,UserInfo_237,ProductInfo_182,UserInfo_238,UserInfo_239,ProductInfo_183,ProductInfo_184,UserInfo_240,ProductInfo_185,UserInfo_241,UserInfo_242,UserInfo_243,UserInfo_244,ProductInfo_186,UserInfo_245,ProductInfo_187,ProductInfo_188,ProductInfo_189,UserInfo_246,UserInfo_247,UserInfo_248,UserInfo_249,ProductInfo_190,ProductInfo_191,ProductInfo_192,ProductInfo_193,ProductInfo_194,UserInfo_250,ProductInfo_195,ProductInfo_196,UserInfo_251,UserInfo_252,UserInfo_253,ProductInfo_197,ProductInfo_198,UserInfo_254,ProductInfo_199,UserInfo_255,ProductInfo_200,ProductInfo_201,ProductInfo_202,ProductInfo_203,UserInfo_256,ProductInfo_204,UserInfo_257,UserInfo_258,UserInfo_259,UserInfo_260,ProductInfo_205,ProductInfo_206,UserInfo_261,UserInfo_262,ProductInfo_207,ProductInfo_208,ProductInfo_209,ProductInfo_210,UserInfo_263,ProductInfo_211,ProductInfo_212,ProductInfo_213,UserInfo_264,UserInfo_265,ProductInfo_214,UserInfo_266,UserInfo_267,ProductInfo_215,ProductInfo_216,UserInfo_268,UserInfo_269,UserInfo_270
count,4000.000000,1.00,119.000000,1515.000000,1515.000000,1515.0,140.000000,1515.000000,1515.0,1515.000000,1515.0,1515.000000,85.000000,11.0,1515.000000,1515.0,1515.000000,116.000000,1515.000000,4000.000000,1515.000000,1515.000000,1515.0,1515.000000,1515.000000,1515.000000,1515.0,1515.000000,1515.00000,8.800000e+02,740.000000,730.000000,26.000000,182.000000,1515.000000,11.000000,116.000000,166.000000,1515.000000,123.000000,1515.0,1515.000000,1515.000000,1515.000000,1515.0,1515.000000,1515.000000,1515.000000,1.515000e+03,1515.000000,1515.0,1515.000000,6.000000,1515.0,1515.000000,166.000000,95.000000,1515.000000,7.000000,1515.000000,1515.0,1515.000000,1515.000000,1515.0,1515.000000,1515.000000,7.000000,1515.000000,1515.000000,152.000000,1515.000000,1515.000000,1515.000000,94.000000,2150.000000,1515.000000,1515.000000,7.000000,1515.0,1515.000000,1515.000000,1515.000000,27.000000,1515.000000,1515.000000,1515.000000,46.000000,1515.000000,4.690000e+02,1515.000000,1515.000000,1515.000000,1515.000000,3.210000e+02,1515.0,1515.000000,1515.000000,14.000000,933.0,1515.000000,...,1515.000000,1515.000000,1515.000000,1515.000000,880.000000,1515.000000,2536.000000,1515.000000,7.000000,1515.000000,1515.000000,1515.000000,1515.000000,1515.00

### 2、数据探索性分析 + 特征工程

In [18]:
# 数据分析往往和特征工程的构造结合在一起

In [26]:
# int64型数据探索
train_B.dtypes[train_B.dtypes == np.int64]

no              int64
flag            int64
UserInfo_170    int64
dtype: object

In [30]:
train_B['UserInfo_170'].describe()
train_B['UserInfo_170'].head()
train_B['UserInfo_170'].unique()

array([0], dtype=int64)

In [ ]:
# 发现UserInfo_170特征只有一个值0,而且是所有的数据都是0,这个特征对任何model都不会带来增益,所以毫无疑问地我们可以将其直接删除.
train_B = train_B.drop('UserInfo_170',axis=1)
train_B.shape

In [35]:
# 针对严重缺失的特征处理
# 我们设立阈值为1%,我们将缺失的特征大于99%的特征删除.(阈值可以自己进行调整).
meaningful_col = []
for col in train_B.columns:
    if train_B[col].count() > train_B.shape[0] * 0.01:
        meaningful_col.append(col)

In [56]:
train_B_1 = train_B[meaningful_col].copy()

In [57]:
train_B.shape
train_B_1.shape

# 我们将数据的维度由490降维至431.既降低了内存消耗,同时后面会发现模型的性能大大提升

(4000, 431)

In [60]:
# 缺失值填充
# 在贷款业务中,很多缺失值是有实际意义的,它们往往来源于用户故意不填写,或者没有,所以我们没法搜集这些数据,这个需要一些简单的业务理解.而针对这样的情况,一个非常有效的方式是直接填充某一个值(可以是-100,999等其他值)[6],此处我们直接用-999进行填充

train_B_1 = train_B_1.fillna(-999)

In [61]:
# 将高线性相关的特征进行删除

train_B_1.corr()

,no,UserInfo_2,UserInfo_3,UserInfo_4,ProductInfo_1,UserInfo_5,UserInfo_6,ProductInfo_2,UserInfo_7,ProductInfo_3,ProductInfo_4,UserInfo_8,ProductInfo_5,ProductInfo_6,UserInfo_10,UserInfo_11,ProductInfo_7,UserInfo_12,ProductInfo_8,ProductInfo_9,ProductInfo_10,UserInfo_13,ProductInfo_11,ProductInfo_12,UserInfo_14,ProductInfo_13,ProductInfo_14,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_19,UserInfo_20,UserInfo_22,UserInfo_23,UserInfo_24,UserInfo_25,ProductInfo_15,ProductInfo_16,ProductInfo_17,UserInfo_26,ProductInfo_18,ProductInfo_19,UserInfo_27,ProductInfo_20,UserInfo_28,ProductInfo_21,ProductInfo_22,ProductInfo_23,UserInfo_30,UserInfo_31,UserInfo_32,UserInfo_33,ProductInfo_24,ProductInfo_25,ProductInfo_26,ProductInfo_27,ProductInfo_28,ProductInfo_29,ProductInfo_30,ProductInfo_31,ProductInfo_32,ProductInfo_33,UserInfo_36,UserInfo_37,ProductInfo_34,UserInfo_38,UserInfo_39,UserInfo_40,ProductInfo_35,ProductInfo_36,ProductInfo_37,UserInfo_42,UserInfo_43,UserInfo_44,ProductInfo_38,UserInfo_46,UserInfo_47,UserInfo_48,ProductInfo_39,UserInfo_49,UserInfo_50,UserInfo_51,ProductInfo_40,ProductInfo_41,UserInfo_52,ProductInfo_42,ProductInfo_43,ProductInfo_44,UserInfo_54,UserInfo_55,UserInfo_56,UserInfo_58,ProductInfo_45,UserInfo_59,UserInfo_60,ProductInfo_46,UserInfo_61,UserInfo_62,UserInfo_63,ProductInfo_47,...,ProductInfo_162,ProductInfo_163,UserInfo_211,UserInfo_212,ProductInfo_164,UserInfo_213,ProductInfo_165,ProductInfo_166,ProductInfo_167,UserInfo_215,UserInfo_216,UserInfo_217,UserInfo_218,UserInfo_219,ProductInfo_168,UserInfo_220,ProductInfo_169,UserInfo_221,ProductInfo_170,ProductInfo_171,UserInfo_222,ProductInfo_172,ProductInfo_173,UserInfo_224,ProductInfo_174,UserInfo_225,UserInfo_226,UserInfo_227,ProductInfo_175,UserInfo_228,ProductInfo_176,UserInfo_229,ProductInfo_177,UserInfo_230,ProductInfo_178,UserInfo_231,UserInfo_232,UserInfo_233,UserInfo_234,ProductInfo_179,ProductInfo_180,UserInfo_235,ProductInfo_181,ProductInfo_182,UserInfo_238,UserInfo_239,ProductInfo_183,ProductInfo_184,ProductInfo_185,UserInfo_241,UserInfo_242,UserInfo_243,ProductInfo_186,ProductInfo_187,ProductInfo_188,ProductInfo_189,UserInfo_246,UserInfo_247,UserInfo_248,UserInfo_249,ProductInfo_190,ProductInfo_191,ProductInfo_192,ProductInfo_193,ProductInfo_194,ProductInfo_195,ProductInfo_196,UserInfo_251,UserInfo_252,UserInfo_253,ProductInfo_197,ProductInfo_198,UserInfo_254,ProductInfo_199,UserInfo_255,ProductInfo_200,ProductInfo_201,ProductInfo_202,ProductInfo_203,UserInfo_256,ProductInfo_204,UserInfo_257,UserInfo_260,ProductInfo_205,UserInfo_261,UserInfo_262,ProductInfo_208,ProductInfo_209,ProductInfo_210,ProductInfo_211,ProductInfo_212,ProductInfo_213,UserInfo_264,UserInfo_265,ProductInfo_214,UserInfo_266,ProductInfo_215,ProductInfo_216,UserInfo_268,UserInfo_269
no,1.000000,-0.009470,0.017689,0.017645,0.000778,0.000450,0.000809,0.000778,0.008722,0.000778,0.000780,0.019377,0.000769,0.000778,0.000760,0.010039,0.000782,0.005970,0.000777,0.000778,0.000778,0.000787,0.000777,0.000777,0.000778,0.000776,0.000769,0.006905,0.007909,-0.001672,-0.005141,0.003592,0.009659,-0.000141,0.000781,0.011761,0.000778,0.000781,0.000779,0.000774,0.000778,0.000778,0.005081,0.000778,-0.007499,0.000777,0.000778,0.000785,0.000778,0.000777,-0.000100,-0.010888,0.000781,0.000777,0.000778,0.000783,0.000778,0.000778,0.000773,0.000780,0.000777,0.000776,0.043201,0.000204,0.000777,0.000772,-0.023536,-0.033093,0.000776,0.000774,0.000778,0.000781,0.000773,-0.013881,0.000779,0.000884,0.000777,-0.017573,0.000777,0.011592,0.000784,0.000797,0.000777,0.000778,-0.004326,0.000778,0.000780,0.000784,0.000910,0.000786,0.017665,-0.017567,0.000778,0.000751,0.004462,0.000778,0.000778,-0.000131,0.005477,0.000779,...,0.000780,0.000785,0.009135,0.000781,0.000773,0.015604,0.000777,0.000776,0.000779,0.016026,0.000802,0.028847,0.016001,-0.023544,0.000777,0.000782,0.000779,0.000778,0.027054,0.000779,0.009319,0.000771,0.000777,0.000791,0.000776,0.000785,0.000800,-0.002519,0.000779,0.000778,0.000773,0.012074,-0.023545

In [63]:
relation = train_B_1.corr()
length = relation.shape[0]
final_cols = []
del_cols = []
for i in range(length):
    if relation.columns[i] not in del_cols:
        final_cols.append(relation.columns[i])
        for j in range(i+1, length):
            if (relation.iloc[i,j] > 0.98) and (relation.columns[j] not in del_cols):
                del_cols.append(relation.columns[j])
            

In [65]:
train_B_1 = train_B_1[final_cols]

### 3、模型训练与测试

In [70]:
# 这里我们直接选用XGBoost
# 为了能够复现成绩,注意在每个算法的包中都有一个随机种子,在XGBoost里面就是random_seed[9],此处我们选择随意一组参数(可以自己调整)

train_B_flag = train_B_1['flag']
train_B_1.drop('no', axis=1, inplace=True)
train_B_1.drop('flag', axis=1, inplace=True)

In [71]:
train_B_1.head()

,UserInfo_2,UserInfo_3,UserInfo_4,ProductInfo_1,UserInfo_5,UserInfo_7,UserInfo_8,UserInfo_11,UserInfo_12,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_19,UserInfo_20,UserInfo_23,UserInfo_25,UserInfo_27,UserInfo_28,UserInfo_33,UserInfo_36,UserInfo_39,UserInfo_40,UserInfo_44,UserInfo_48,UserInfo_49,UserInfo_52,UserInfo_54,UserInfo_60,UserInfo_63,UserInfo_66,ProductInfo_49,ProductInfo_50,UserInfo_67,UserInfo_68,WebInfo_1,UserInfo_72,UserInfo_73,UserInfo_75,UserInfo_76,UserInfo_80,UserInfo_82,UserInfo_83,UserInfo_85,UserInfo_92,UserInfo_94,UserInfo_96,UserInfo_97,UserInfo_100,UserInfo_101,UserInfo_104,UserInfo_105,UserInfo_106,UserInfo_107,UserInfo_108,ProductInfo_89,UserInfo_113,UserInfo_115,UserInfo_118,UserInfo_121,UserInfo_122,UserInfo_125,UserInfo_127,UserInfo_130,UserInfo_132,UserInfo_133,UserInfo_134,UserInfo_135,UserInfo_136,UserInfo_147,UserInfo_149,UserInfo_153,UserInfo_155,UserInfo_160,UserInfo_169,UserInfo_179,WebInfo_3,UserInfo_184,UserInfo_186,UserInfo_190,UserInfo_193,UserInfo_196,UserInfo_197,UserInfo_200,UserInfo_210,UserInfo_217,UserInfo_222,UserInfo_227,UserInfo_229,UserInfo_231,UserInfo_238,UserInfo_242,UserInfo_243,UserInfo_246,UserInfo_248,UserInfo_261,UserInfo_262,UserInfo_269
0,5226.59,0.0,0.0,0.0,296.0,0.0,-999.0,20.0,2.0,-999.0,-999.0,5226.59,5463.65,5226.59,4.0,-999.0,35360.16,0.0,0.0,-999.0,0.0,1.0,0.0,-999.0,-999.0,131379.6,0.0,0.0,5226.59,0.0,0.0,-999.0,-999.0,0.0,-999.0,-999.0,1798.16,-999.0,0.0,5226.59,74.0,0.0,-999.0,-20009.32,0.0,33009.63,4683.03,-999.00,2977.01,9630.4,-999.0,0.0,-999.0,-999.0,5226.59,-999.0,1.0,0.0,-999.0,1.0,0.00,0.0,-999.0,0.0,0.0,2450.53,0.0,31.0,26900.0,0.0,0.0,0.0,-999.0,-999.0,33009.63,-999.0,33009.63,0.0,0.0,-999.0,0.00,-999.0,1.0,-999.0,-999.0,0.0,0.0,0.0,-999.0,0.0,10000.0,2.0,-999.0,0.00,-999.0,1.0,2940.0
1,0.00,0.0,0.0,0.0,-999.0,46000.0,-999.0,-999.0,1.0,0.0,-999.0,736.00,3064.50,1472.00,-999.0,-999.0,32382.47,0.0,-999.0,-999.0,-999.0,5.0,0.0,-999.0,-999.0,0.0,-999.0,200000.0,736.00,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1094.41,-999.0,-999.0,736.00,62.0,0.0,-999.0,738.38,0.0,1472.00,1448.61,31901.59,0.00,0.0,-999.0,0.0,0.0,2.0,736.00,-999.0,1.0,0.0,-999.0,-999.0,2603.53,0.0,39.0,0.0,0.0,0.00,0.0,-999.0,-999.0,1.0,0.0,0.0,-999.0,707609.0,1472.00,-999.0,1472.00,0.0,-999.0,-999.0,621.45,14084.0,1.0,-999.0,-999.0,3.0,0.0,736.0,-999.0,0.0,86500.0,0.0,-999.0,30910.47,-999.0,2.0,0.0
2,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2.0,-999.0,-999.0,-999.00,-999.00,-999.00,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,2.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.00,37.0,-999.0,-999.0,-999.00,-999.0,-999.00,-999.00,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,2.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.00,-999.0,-999.0,-999.0,-999.00,-999.0,1.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,14072.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0
3,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2.0,-999.0,-999.0,-999.00,-999.00,-999.00,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.00,-999.0,-999.00,-999.00,-999.00,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,2.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.00,-999.0,-999.0,-999.0,-999.00,-999.0,1.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0
4,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2.0,-999.0,-999.0,-999.00,-999.00,-999.00,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,3.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.00,-999.0,-999.0,-999.0,-999.0,-999.0

In [72]:
# xgboost模型
dtrain_B = xgb.DMatrix(data=train_B_1, label=train_B_flag)
Trate=0.25  
params = {'booster':'gbtree',
                  'eta': 0.1,
                   'max_depth': 4,
                   'max_delta_step': 0,
                   'subsample':0.9,      
                   'colsample_bytree': 0.9,
                   'base_score': Trate,
                   'objective': 'binary:logistic',
                   'lambda':5,
                   'alpha':8,
                   'random_seed':100
                   }
params['eval_metric'] = 'auc' 
xgb_model = xgb.train(params, dtrain_B, num_boost_round=200, maximize=True, verbose_eval=True )


In [73]:
# 模型测试

res = xgb_model.predict(xgb.DMatrix(test_B[train_B_1.columns].fillna(-999)))
test_B['pred'] = res
test_B[['no', 'pred']].to_csv('submit.csv',index = None)

In [74]:
# 线上AUC = 0.602393

## 测试LGB

In [76]:
import lightgbm as lgb
def lgb_feature_selection(tr_x, tr_y, model_seed =666,num_rounds = 500):
    lgb_tr = lgb.Dataset(tr_x, tr_y)
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'random_state': model_seed}
    model = lgb.train(lgb_params, lgb_tr, num_boost_round=num_rounds,verbose_eval=100)
    return model

In [77]:
f_model = lgb_feature_selection(train_B_1, train_B_flag)


In [81]:
lgb_pred = f_model.predict(test_B[train_B_1.columns].fillna(-999))
test_B['pred'] = lgb_pred
test_B[['no', 'pred']].to_csv('submitLGB.csv',index = None)

In [82]:
# 线上AUC = 0.571084
# 应该是参数的问题，导致lgb测评小于xgboost

In [84]:
# 改变参数
def lgb_feature_selection2(tr_x, tr_y, model_seed =666,num_rounds = 500):
    lgb_tr = lgb.Dataset(tr_x, tr_y)
    lgb_params = {
         'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'max_depth':17,
    'feature_fraction':0.80,
    'lambda_l1':0.6,
#     'scale_pos_weight':1.1,
    'random_state': 0}
    model = lgb.train(lgb_params, lgb_tr, num_boost_round=num_rounds,verbose_eval=100)
    return model
f_model = lgb_feature_selection2(train_B_1, train_B_flag)
lgb_pred = f_model.predict(test_B[train_B_1.columns].fillna(-999))
test_B['pred'] = lgb_pred
test_B[['no', 'pred']].to_csv('submitLGB.csv',index = None)

In [ ]:
# 线上AUC = 0.580101